In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

def get_page_text(url, user_agent=USER_AGENT, error_log=True):
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers, allow_redirects=True)

    if response.status_code == 200:
        page_text = response.text
        return page_text
    elif error_log:
        print(f'Ошибка! Response code: {response.status_code}')
        print(f'URL: {url}')
        print('--------------')
    
    return None

In [4]:
index = 1
url = f'https://em-russia.ru/shop/all/?PAGEN_1={index}&SIZEN_1=16'

page_text = get_page_text(url, USER_AGENT)

In [5]:
url

'https://em-russia.ru/shop/all/?PAGEN_1=1&SIZEN_1=16'

In [6]:
print(page_text[:1000])

<!doctype html>
<html>
<head>
    <!-- Yandex.Metrika counter -->
<script type="text/javascript" >
   (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};
   m[i].l=1*new Date();
   for (var j = 0; j < document.scripts.length; j++) {if (document.scripts[j].src === r) { return; }}
   k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})
   (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");

   ym(96850838, "init", {
        clickmap:true,
        trackLinks:true,
        accurateTrackBounce:true,
        webvisor:true
   });
</script>
<noscript><div><img src="https://mc.yandex.ru/watch/96850838" style="position:absolute; left:-9999px;" alt="" /></div></noscript>
<!-- /Yandex.Metrika counter -->

    <meta property="og:type" content="website"/>
    <meta property="og:site_name" content="em-russia.ru"/>
    <meta property="og:title" content=""/>
    <meta property="og:url" content="htt

In [7]:
products = pd.DataFrame(columns=['title', 'url', 'price', 'description'])

breakpoint = 0

for page in (1,2,3):

    print('dowload page:', page)
    url = f'https://em-russia.ru/shop/all/?PAGEN_1={page}&SIZEN_1=16'

    page_text = get_page_text(url, USER_AGENT)

    soup = BeautifulSoup(page_text)
    posts_list = soup.find_all('div', {'class': 'product__title'})   
    for p in posts_list:
        
        lnk = p.find('a').attrs['href']
        title = re.sub('[^А-ЯЁа-яё0-9\s]', ' ', p.text)
        title = re.sub('\s\s+', ' ', title)
        
        page_url = 'https://em-russia.ru' + lnk
        clean_path = '/'.join([d for d in page_url.split('/')[2:] if len(d) > 0])
        
        page_text = get_page_text(page_url, USER_AGENT)
        
        if page_text is None:
            continue
        
        description = ''
        product_page = BeautifulSoup(page_text)
        main_div = product_page.find('div', {'id': 'main'})
        
        cd = main_div.find('div', {'class': 'container'}, recursive=False) 
        
        desc = cd.find('div')
        
        if desc is not None:
            description = description + desc.text.strip()       
        else:
            print(desc)

        tabs = cd.find_all('div', {'class': 'product-nav-tabs__button'})
        
        if tabs is None:
            continue
        else:
            tabs_content = cd.find_all('div', {'class': 'product-tab'})
            for t, tc in zip(tabs, tabs_content):
                description = description + '\n' + t.text.strip() + '\r' +tc.text.strip()
        
        
        description = re.sub('[\n\r]+', '\n', description)
        description = re.sub('[\t]+', ' ', description)
        price = main_div.find('span', {'id': 'item-price'})
        price = re.sub('[^0-9\.\s]', ' ', price.text.strip())
        
        
        products.loc[ len(products.index )] = [title, page_url, price, description]
            
        #breakpoint = breakpoint+1
        #if breakpoint>5:
        #    break 
            



dowload page: 1
dowload page: 2
dowload page: 3


In [8]:
print(products.iloc[2]['description'])

Набор для самостоятельного активирования (приготовления) 3-х литров ЭМ-БИО («ЕМ·1 микробиологическое удобрение «Восток ЭМ-1») микробиологического удобрения для всех видов садовых и комнатных растений.
  Концентрат ЭМ-БИО (ЕМ·1 микробиологическое удобрение «Восток ЭМ-1») представляет собой концентрированную культуру эффективных микроорганизмов, находящуюся в устойчивом неактивном состоянии и предназначенную для длительного хранения, визуально это – светло-коричневая жидкость с приятным кисломолочным запахом.
  Срок годности: 12 месяцев с даты изготовления
Состав
В набор входит:
концентрат ЭМ-БИО (ЕМ·1 микробиологическое удобрение «Восток ЭМ-1») в количестве  30 мл (3 ампулы по 10 мл)
патока тростниковая 30 мл (3 ампулы по 10 мл) 
инструкция по разведению и применению. 
  СОСТАВ концентрата: вода питьевая, патока, комплекс молочнокислых бактерий, дрожжей и продуктов их жизнедеятельности. В 1 см3 содержится не менее 1*106 КОЕ молочнокислых бактерий и 1*104 КОЕ дрожжей.
Применение
Продукт 

In [9]:
products.to_csv('data.csv')

In [10]:
products.describe()

,title,url,price,description
count,45,45,45,45
unique,32,45,38,37
top,ЭМ БИО удобрение 1 Восток ЭМ 1,https://em-russia.ru/shop/all/vozrozhdennoe-bu...,5 380 .,Устраняет неприятные запахи (в том числе запах...
freq,5,1,3,4
